In [ ]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [ ]:
# link to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
links = np.array(pd.read_csv('/content/drive/MyDrive/ROI_data.csv')) # read the ROI return ratio for each platform
links

array([['ROI', 0.031, 0.049, 0.024, 0.039, 0.016, 0.024, 0.046, 0.026,
        0.033, 0.044],
       ['Second Firms ROI Estimate', 0.049, 0.023, 0.024, 0.039, 0.044,
        0.046, 0.026, 0.019, 0.037, 0.026]], dtype=object)

In [ ]:
index = np.array(pd.read_csv('/content/drive/MyDrive/index_data.csv')) # read the index 
index

array([['January', 1, 13, 25, 37, 49, 61, 73, 85, 97, 109],
       ['February', 2, 14, 26, 38, 50, 62, 74, 86, 98, 110],
       ['March', 3, 15, 27, 39, 51, 63, 75, 87, 99, 111],
       ['April', 4, 16, 28, 40, 52, 64, 76, 88, 100, 112],
       ['May', 5, 17, 29, 41, 53, 65, 77, 89, 101, 113],
       ['June', 6, 18, 30, 42, 54, 66, 78, 90, 102, 114],
       ['July', 7, 19, 31, 43, 55, 67, 79, 91, 103, 115],
       ['August', 8, 20, 32, 44, 56, 68, 80, 92, 104, 116],
       ['September', 9, 21, 33, 45, 57, 69, 81, 93, 105, 117],
       ['October', 10, 22, 34, 46, 58, 70, 82, 94, 106, 118],
       ['November', 11, 23, 35, 47, 59, 71, 83, 95, 107, 119],
       ['December', 12, 24, 36, 48, 60, 72, 84, 96, 108, 120]],
      dtype=object)

In [ ]:
mat = np.array(pd.read_csv('/content/drive/MyDrive/roi_mat.csv')) # read the ROI return ratio for each platform over the year
mat

array([['January', 4.0, 3.6, 2.4, 3.9, 3.0, 3.5, 3.6, 2.25, 3.5, 3.5],
       ['February', 4.0, 3.9, 2.7, 3.8, 4.3, 3.2, 2.7, 1.8, 3.7, 3.5],
       ['March', 3.5, 2.9, 3.1, 3.8, 2.4, 4.1, 3.7, 2.6, 4.2, 2.5],
       ['April', 3.8, 3.1, 2.4, 4.4, 2.4, 3.8, 3.7, 2.5, 3.6, 2.9],
       ['May', 3.5, 3.2, 1.9, 3.4, 2.7, 2.7, 3.9, 2.2, 4.5, 3.9],
       ['June', 4.0, 3.2, 2.7, 3.4, 3.4, 3.0, 4.5, 2.1, 3.8, 4.1],
       ['July', 3.9, 3.6, 2.0, 4.4, 3.9, 3.7, 4.3, 1.8, 4.0, 3.8],
       ['August', 4.2, 3.3, 2.8, 4.2, 2.0, 3.7, 3.6, 1.5, 4.4, 4.3],
       ['September', 4.1, 2.8, 2.5, 4.2, 2.9, 3.7, 2.8, 2.5, 4.0, 3.4],
       ['October', 3.0, 3.0, 3.1, 4.6, 3.1, 3.3, 3.2, 2.3, 2.5, 3.2],
       ['November', 4.8, 3.3, 2.7, 4.1, 2.9, 3.6, 4.2, 3.0, 3.1, 4.1],
       ['December', 4.8, 4.0, 1.9, 3.7, 4.2, 3.6, 2.6, 2.9, 3.6, 3.7]],
      dtype=object)

Find the best budget allocation using first ROI return ratio

In [ ]:
obj = np.array(links[0,1:]) # get the objective ROI for each platform
A = np.zeros((3,10))
A[0,:] = [1,1,0,0,-1,0,0,0,0,-1] # the first contraint: x1 + x2 - x5 - x10 <= 0
A[1,:] = [0,0,2,2,-1,-1,-1,-1,-1,0] # the social media constraint: 2x3 + 2x4 - x5 - x6 - x7  - x8 - x9 ≤  0
A[2,:] = [1,1,1,1,1,1,1,1,1,1] # the budget constraint: x1 + x2 + x3 + x4 + x5 + x6 - x7 - x8 + x9 + x10 ≤ 10M
b = np.array([0,0,10]) 

In [ ]:
ojModel = gp.Model() # initialize an empty model

ojModX = ojModel.addMVar(10, ub = 3) # tell the model there are 10 variables and upper bound is 3M
ojModCon = ojModel.addConstr(A @ ojModX <= b) # add the constraints to the model
ojModel.setObjective(obj @ ojModX,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

ojModel.Params.OutputFlag = 0 # tell gurobi to shut up!!
ojModel.Params.TimeLimit = 3600

In [ ]:
ojModel.optimize() # solve the LP

The budget allocation based on first ROI

In [ ]:
ojModX.x # the optimal allocation

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

In [ ]:
ojModel.objVal # the objective value

0.45600000000000007

Using the second ROI to find the best budget allocation

In [ ]:
obja = np.array(links[1,1:]) # get the second objective ROI for each platform
obja

array([0.049, 0.023, 0.024, 0.039, 0.044, 0.046, 0.026, 0.019, 0.037,
       0.026], dtype=object)

In [ ]:
ojaModel = gp.Model() # initialize an empty model

ojaModX = ojaModel.addMVar(10, ub = 3) # tell the model there are 10 variables and 3 constraints
ojaModCon = ojaModel.addConstr(A @ ojaModX <= b) # add the constraints to the model
ojaModel.setObjective(obja @ ojaModX,sense=gp.GRB.MAXIMIZE) # add the objective to the model

ojaModel.Params.OutputFlag = 0 # tell gurobi to shut up!!
ojaModel.Params.TimeLimit = 3600

In [ ]:
ojaModel.optimize()
ojaModX.x  # budget allocation based on second ROI

array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.])

In [ ]:
ojaModel.objVal # the objetive value based on second ROI

0.45600000000000007

Using first ROI and second allocation

In [ ]:
obj @ ojaModX.x

0.252

Using second ROI and first allocation

In [ ]:
obja @ ojModX.x

0.264

Lower range of obj where optimal corner stay the same

In [ ]:
ojModX.SAObjLow

array([ -inf, 0.039,  -inf, 0.033,  -inf,  -inf, 0.039,  -inf,  -inf,
       0.029])

Upper range of obj where optimal corner stay the same

In [ ]:
ojModX.SAObjUp

array([0.049, 0.062, 0.039, 0.046, 0.029, 0.039,   inf, 0.039, 0.039,
         inf])

# Question 7

In [ ]:
matt = np.array(0.01*mat[0:,1:]) # getting an array for all the ROI in each platorm throughout the year in percent ratio
matt

array([[0.04, 0.036000000000000004, 0.024, 0.039, 0.03, 0.035,
        0.036000000000000004, 0.0225, 0.035, 0.035],
       [0.04, 0.039, 0.027000000000000003, 0.038, 0.043, 0.032,
        0.027000000000000003, 0.018000000000000002, 0.037000000000000005,
        0.035],
       [0.035, 0.028999999999999998, 0.031000000000000003, 0.038, 0.024,
        0.040999999999999995, 0.037000000000000005, 0.026000000000000002,
        0.042, 0.025],
       [0.038, 0.031000000000000003, 0.024, 0.044000000000000004, 0.024,
        0.038, 0.037000000000000005, 0.025, 0.036000000000000004,
        0.028999999999999998],
       [0.035, 0.032, 0.019, 0.034, 0.027000000000000003,
        0.027000000000000003, 0.039, 0.022000000000000002, 0.045, 0.039],
       [0.04, 0.032, 0.027000000000000003, 0.034, 0.034, 0.03, 0.045,
        0.021, 0.038, 0.040999999999999995],
       [0.039, 0.036000000000000004, 0.02, 0.044000000000000004, 0.039,
        0.037000000000000005, 0.043, 0.018000000000000002, 0.04, 0.038]

In [ ]:
npages = matt.shape[0] # see how many months it has
npages

12

In [ ]:
P = np.zeros((npages,10)) # create an empty array to store the best budget allocation for
P

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
default_budget = 10 # the default budget at the start of each month
reinvest_ratio = 0.5

In [ ]:
b = np.array([0,0,default_budget])
for i in range(npages):
  obj = np.array(matt[i,:])
  ojModel = gp.Model()
  ojModX = ojModel.addMVar(10, ub = 3)
  ojModCon = ojModel.addConstr(A @ ojModX <= b)
  ojModel.setObjective(obj @ ojModX,sense=gp.GRB.MAXIMIZE)
  ojModel.Params.OutputFlag = 0 # tell gurobi to shut up!!
  ojModel.Params.TimeLimit = 3600
  ojModel.optimize()
  P[i] = ojModX.x
  b = np.array([0,0,default_budget + reinvest_ratio * ojModel.objVal])
  print("the profit of that month: ", ojModel.objVal)
  print("the budget of ", i + 1, "th month: ",b[2])
  print()
  

the profit of that month:  0.373
the budget of  1 th month:  10.1865

the profit of that month:  0.40629600000000005
the budget of  2 th month:  10.203148

the profit of that month:  0.407516476
the budget of  3 th month:  10.203758238

the profit of that month:  0.40033529656800004
the budget of  4 th month:  10.200167648284

the profit of that month:  0.41100586768993996
the budget of  5 th month:  10.20550293384497

the profit of that month:  0.4238091114861089
the budget of  6 th month:  10.211904555743054

the profit of that month:  0.4282642776739791
the budget of  7 th month:  10.21413213883699

the profit of that month:  0.4379935498311536
the budget of  8 th month:  10.218996774915578

the profit of that month:  0.40271237905933416
the budget of  9 th month:  10.201356189529667

the profit of that month:  0.37144339806494936
the budget of  10 th month:  10.185721699032475

the profit of that month:  0.4416145896603315
the budget of  11 th month:  10.220807294830166

the profit

In [ ]:
for i in range(npages):
  print("the #", i+1, "month's allocation")
  print(P[i])
  print()

the # 1 month's allocation
[3.         0.         0.         1.33333333 0.         0.
 2.66666667 0.         0.         3.        ]

the # 2 month's allocation
[3.     0.     0.     2.3955 3.     0.     0.     0.     1.791  0.    ]

the # 3 month's allocation
[0.       0.       0.       3.       0.       3.       1.203148 0.
 3.       0.      ]

the # 4 month's allocation
[0.         0.         0.         3.         0.         3.
 3.         0.         1.20375824 0.        ]

the # 5 month's allocation
[1.20016765 0.         0.         0.         0.         0.
 3.         0.         3.         3.        ]

the # 6 month's allocation
[3.         0.         0.         0.         0.         0.
 3.         0.         1.20550293 3.        ]

the # 7 month's allocation
[0.         0.         0.         3.         1.21190456 0.
 3.         0.         3.         0.        ]

the # 8 month's allocation
[2.71413214 0.         0.         1.5        0.         0.
 0.         0.         3.         